In [1]:
from IPython.display import display, Markdown
import math
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import accuracy_score

Task 1

In [2]:
data = np.array(pd.read_csv('Data/data.csv', header=None))
labels = np.ravel(pd.read_csv('Data/label.csv', header=None))

In [3]:
def sim_type(X, similarity):
    if similarity == 'cosine':
        X = pairwise_distances(X, metric='cosine')
    elif similarity == 'jaccard':
        X = pairwise_distances(X, metric='hamming')
    return X

class KMeans:
    def __init__(self, k, similarity='euclidean', stop='none') -> None:
        self.k = k
        self.similarity = similarity
        self.stop = stop
        self.centroids = None
        self.c_labels = None
        self.prev_score = 0.0
        self.sse = 0.0
        self.num_iter = 0

    def calc_sse(self, X, y):
        for i, centroid in enumerate(self.centroids):
            cluster_points = X[y == i]
            self.sse += np.sum((cluster_points - centroid) ** 2)
        return self.sse
    
    def cluster_labels(self, temp, true):
        c_labels = np.zeros(len(temp), dtype=int)
        for cluster in range(100):
            indices = np.where(temp == cluster)[0]
            c_labels[indices] = np.argmax([np.sum(true[indices] == label) for label in np.unique(true)])
        return c_labels

    def fit(self, X, y, iterations=100):
        # pair_dist = sim_type(X, self.similarity)
        n_samples, n_features = X.shape
        self.centroids = X[np.random.choice(n_samples, self.k, replace=False)]
        temp = []

        for _ in range(iterations):
            temp = []
            for point in X:
                distances = np.sqrt(np.sum((self.centroids - point)**2, axis=1))
                temp.append(np.argmin(distances))
            temp = np.asarray(temp)
            
            new_centroids = np.array([np.mean(X[temp == i], axis=0) for i in range(self.k)])

            if self.stop == 'increase':
                self.sse = self.calc_sse(X, temp)
                if self.sse > self.prev_score:
                    break
                self.prev_score = self.sse
            elif self.stop == 'iter':
                continue
            elif np.max(np.abs(self.centroids - new_centroids)) < 0.001:
                break

            self.centroids = new_centroids
            self.num_iter += 1

        self.sse_score = self.calc_sse(X, temp)
        self.c_labels = self.cluster_labels(temp, y)
        return temp

In [4]:
euclidean = KMeans(k=len(np.unique(labels)))
euclidean_labels = euclidean.fit(data, labels)
print("Iterations:", euclidean.num_iter)

Iterations: 43


In [5]:
cosine = KMeans(k=len(np.unique(labels)))
cosine_nums = pairwise_distances(data, metric='cosine')
cosine_labels = cosine.fit(cosine_nums, labels)
print("Iterations:", cosine.num_iter)

Iterations: 88


In [6]:
jaccard = KMeans(k=len(np.unique(labels)))
jaccard_nums = pairwise_distances(data, metric='hamming')
jaccard_labels = jaccard.fit(jaccard_nums, labels)
print("Iterations:", jaccard.num_iter)

Iterations: 18


In [7]:
print("Euclidean SSE:", euclidean.sse)
print("Cosine SSE:", cosine.sse)
print("Jaccard SSE:", jaccard.sse)

Euclidean SSE: 25324912667.12255
Cosine SSE: 677753.4069187804
Jaccard SSE: 33035.760057596606


Q1. After comparing the SSEs of Euclidean, Cosine, and Jaccard distances, we see see that Jaccard is the best as it has the lowest SSE.

In [8]:
print("Euclidean Accuracy:", accuracy_score(labels, euclidean.c_labels))
print("Cosine Accuracy:", accuracy_score(labels, cosine.c_labels))
print("Jaccard Accuracy:", accuracy_score(labels, jaccard.c_labels))

Euclidean Accuracy: 0.6011
Cosine Accuracy: 0.5254
Jaccard Accuracy: 0.3403


Q2. Comparing the Accuracys of Euclidean, Cosine, and Jaccard, we see see that Euclidean has the highest accuracy.

In [16]:
increased_euclidean = KMeans(k=len(np.unique(labels)), stop='increase')
euclidean_labels = increased_euclidean.fit(data, labels)

In [17]:
increased_cosine = KMeans(k=len(np.unique(labels)), stop='increase')
cosine_labels = increased_cosine.fit(cosine_nums, labels)

In [18]:
increased_jaccard = KMeans(k=len(np.unique(labels)), stop='increase')
jaccard_labels = increased_jaccard.fit(jaccard_nums, labels)

In [12]:
euclidean = KMeans(k=len(np.unique(labels)), stop='iter')
euclidean_labels = euclidean.fit(data, labels)

In [13]:
cosine = KMeans(k=len(np.unique(labels)), stop='iter')
cosine_labels = cosine.fit(cosine_nums, labels)

In [14]:
jaccard = KMeans(k=len(np.unique(labels)), stop='iter')
jaccard_labels = jaccard.fit(jaccard_nums, labels)

Q3. The method that requires the most time and iterations to converge is when we allow the max number of iterations to complete. 

In [19]:
print("Euclidean SSE with Increases in SSE:", increased_euclidean.sse)
print("Cosine SSE with Increases in SSE:", increased_cosine.sse)
print("Jaccard SSE with Increases in SSE:", increased_jaccard.sse)

Euclidean SSE with Increases in SSE: 90548946962.0
Cosine SSE with Increases in SSE: 2326627.814077913
Jaccard SSE with Increases in SSE: 134101.51697534873


Q4. Similar to the first question, the Jaccard SSE is the best even after the SSE increases in the next iteration.

Q5. To summarize, Jaccard had the best SSE when we don't change the centroid position or after having the SSE increase in the next iteration.
However, the Euclidean distance had the best Accuracy when there is no change in centroid position. Finally, among the three methods, the one
that required the most time and iterations to converge is when we allow the max number of iterations to complete.